In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import math
%matplotlib inline

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)   
    
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):

    for line in lines:
        for x1,y1,x2,y2 in line:
            
            slope = ((y2-y1)/(x2-x1))
            intercept = ((y1 - (slope * x1)) + (y2 - (slope * x2))) / 2
            
            # slope > 0 -> right lane
            if slope > 0: 
                right_slopes.append(slope)
                right_intercepts.append(intercept)
                right_ys.append((y1 + y2) / 2)
            
            # slope < 0 -> left lane
            if slope < 0:
                left_slopes.append(slope)
                left_intercepts.append(intercept)                
                left_ys.append(y1)
                left_ys.append(y2)
                
    N = 50                
    min_y = min(min(right_ys[len(right_ys) - N: len(right_ys)]), min(left_ys[len(left_ys) - N: len(left_ys)]))
    max_y = img.shape[0]
    
    mean_right_slope = np.mean(right_slopes[len(right_slopes) - N: len(right_slopes)])
    mean_right_intercept = np.mean(right_intercepts[len(right_intercepts) - N: len(right_intercepts)])
    
    r_a_x = (min_y - mean_right_intercept) / mean_right_slope 
    r_b_x = (max_y - mean_right_intercept) / mean_right_slope    
    
    cv2.line(img, (int(r_a_x), int(min_y)), (int(r_b_x), int(max_y)), [255, 0, 0], 15)

    
    mean_left_slope = np.mean(left_slopes[len(left_slopes) - N: len(left_slopes)])
    mean_left_intercept = np.mean(left_intercepts[len(left_intercepts) - N: len(left_intercepts)])
    
    l_a_x = (min_y - mean_left_intercept) / mean_left_slope
    l_b_x = (max_y - mean_left_intercept) / mean_left_slope
    
    cv2.line(img, (int(l_a_x), int(min_y)), (int(l_b_x), int(max_y)), [255, 0, 0], 15)


def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    return cv2.addWeighted(initial_img, α, img, β, λ)

right_slopes = []
right_intercepts = []
right_ys = []
left_slopes = []
left_intercepts = []
left_ys = []


def process_image(image):
    # Gray scale
    gray = grayscale(image)
    
    # Gaussian smoothing
    kernel_size = 7
    blur_gray = gaussian_blur(gray, kernel_size)
    
    # Canny
    low_threshold = 160
    high_threshold = 180
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    # Mask
    imshape = image.shape
    vertices = np.array([[(130, imshape[0]),(450, 320), (530, 320), (imshape[1], imshape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
    
    # Hough
    rho = 1
    theta = np.pi/150
    threshold = 10
    min_line_length = 20
    max_line_gap = 5

    hough_img = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)

    result = weighted_img(hough_img, image)
    
    return result


# Process solidWhiteRight.mp4 
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)


# Process solidYellowLeft.mp4
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

## Reflections

1. Filtering of line segments should become more extensive. At the moment only horizontal lines with slope 0 are filtered out. Having filtering logic using some predefined thresholds could help to further reduce noise.
2. Mean values of slope, intercept and y could be calculated based on the line segments stored globally on each frame.
